In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
#x,y,c,
poseName = 'Tendu'
Coords = np.load("RootCoordinates/"+poseName+"X.npy")
print(Coords)
print(len(Coords))

[[0.51232  0.276488 0.874558 ... 0.520434 0.758888 0.839005]
 [0.514263 0.276551 0.882466 ... 0.522431 0.758982 0.824334]
 [0.518342 0.276565 0.868114 ... 0.516298 0.759002 0.815286]
 ...
 [0.493936 0.301091 0.84863  ... 0.518592 0.731722 0.788134]
 [0.493955 0.298433 0.845354 ... 0.518581 0.731681 0.797879]
 [0.495964 0.298396 0.850061 ... 0.51852  0.731617 0.805679]]
870


In [3]:
#Helper Methods:
import math
def calcDistance(x1,y1,x2,y2):
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

In [4]:
#Get the Mean Skeleton's coordinates
PoseMeanX = np.load("Stats/"+poseName+"MeanX.npy")
print(PoseMeanX)
print(len(PoseMeanX))
PoseMeanY = np.load("Stats/"+poseName+"MeanY.npy")
print(PoseMeanY)
print(len(PoseMeanY))
MeanTuples = list(zip(PoseMeanX, PoseMeanY))
print(MeanTuples)

[0.50524143 0.50119931 0.47309403 0.45067261 0.45399262 0.5293227
 0.57912379 0.62074823 0.5        0.48529658 0.50830061 0.51413092
 0.51451088 0.48642372 0.45788078 0.49788841 0.51196986 0.485917
 0.48696091 0.44570213 0.45023933 0.45547568 0.4879609  0.49050958
 0.52061475]
25
[-0.29520034 -0.35531341 -0.35187719 -0.28127309 -0.21523528 -0.35939023
 -0.37630604 -0.38268153 -0.5        -0.49991867 -0.60850739 -0.72099211
 -0.5006225  -0.60642855 -0.72161289 -0.2877979  -0.28811658 -0.30094813
 -0.28331807 -0.76354157 -0.76108922 -0.7349929  -0.73847128 -0.73327875
 -0.73400667]
25
[(0.5052414275862066, -0.29520033793103445), (0.5011993068965518, -0.355313409195402), (0.47309402528735606, -0.3518771908045982), (0.4506726149425292, -0.28127308620689656), (0.4539926172413798, -0.21523528160919533), (0.529322697701149, -0.3593902264367813), (0.5791237896551722, -0.3763060425287358), (0.6207482310344831, -0.38268152988505794), (0.5, -0.5), (0.48529658275862064, -0.4999186689655177), (0.50

In [5]:
POSE = "Tendu"
allDistanceVectors = []
for sample in Coords:
    xCoords = sample[0::3]
    #print(xCoords)
    yPos = sample[1::3]
    yCoords = [-y for y in yPos] # need to make it negative to be consistent - the mean skeleton's y saved the negative coords.
    SampleTuples = list(zip(xCoords, yCoords))
    #print(SampleTuples)
    longZip = list(zip(MeanTuples, SampleTuples))
    #print(longZip)
    distanceVector = []
    for ((x1,y1),(x2,y2)) in longZip:
        d = calcDistance(x1,y1,x2,y2)
        #print(d)
        distanceVector.append(d)
    allDistanceVectors.append(distanceVector)

print(allDistanceVectors)
print(len(allDistanceVectors))
print(len(allDistanceVectors[0]))
np.save("DataForSVM/"+POSE+"DistanceVectorsPerSample.npy", allDistanceVectors)

[[0.020006443418622535, 0.01103130883896824, 0.012664991056362724, 0.02523551856756825, 0.02509824293764062, 0.013142355238286671, 0.012873404472326998, 0.015177503400420466, 0.0, 0.00043330901717341617, 0.016944698356392236, 0.021876844475770166, 0.002223315943835841, 0.018385287337677105, 0.024568419515981668, 0.02927494799823221, 0.0280186270955726, 0.022547535051535613, 0.03965031149035178, 0.03632927223424981, 0.03448357084243952, 0.026184834385314906, 0.02527527270813667, 0.02766573203633072, 0.024881987549274943], [0.020716818628429345, 0.010618482865615357, 0.012644869099109389, 0.02466464854656986, 0.02998462140399201, 0.011990184321347722, 0.012854088337513073, 0.016384034047293647, 0.0, 0.002414996788935915, 0.01726813617782149, 0.021677497147190736, 0.0006229506151140245, 0.015180340676495641, 0.01943694485585014, 0.029992377253867126, 0.02796842356669057, 0.0255324183535949, 0.041557140271154334, 0.03618025756081326, 0.0288014070182251, 0.02218044795719364, 0.0246917539269

In [6]:
#These features are mixing coordinates and distances: 100 features per sample: 75 + 25
PoseAugFeaturesX = []
for i in range(len(Coords)):
    newList = list(Coords[i])+list(allDistanceVectors[i])
    PoseAugFeaturesX.append(newList)
    
print(PoseAugFeaturesX[0])
print(len(PoseAugFeaturesX[0]))
np.save("DataForSVM/"+POSE+"AugMeanDistFeaturesX.npy", PoseAugFeaturesX)

[0.51232, 0.27648800000000007, 0.874558, 0.504157, 0.34468600000000005, 0.8967400000000001, 0.47350899999999996, 0.33921900000000005, 0.837723, 0.45906800000000003, 0.25747500000000006, 0.9190809999999999, 0.46331100000000003, 0.19193100000000007, 0.861468, 0.534726, 0.34741000000000005, 0.862095, 0.589901, 0.36926500000000007, 0.822382, 0.634819, 0.3883710000000001, 0.862644, 0.5, 0.5, 0.777836, 0.48572099999999996, 0.5000060000000001, 0.729884, 0.510255, 0.625339, 0.842876, 0.51034, 0.742538, 0.847115, 0.512362, 0.500052, 0.755038, 0.47755800000000004, 0.6225350000000001, 0.825326, 0.445, 0.742534, 0.7711509999999999, 0.5081249999999999, 0.2603710000000001, 0.9328219999999999, 0.524428, 0.26302000000000003, 0.888608, 0.491853, 0.27919600000000006, 0.9162300000000001, 0.526591, 0.282052, 0.544465, 0.442905, 0.799763, 0.686396, 0.44079399999999996, 0.794254, 0.643246, 0.444857, 0.758928, 0.749522, 0.477557, 0.761506, 0.611578, 0.479664, 0.75873, 0.6034079999999999, 0.520434, 0.758888, 